### Installations

In [1]:
#choose a minimal A100 GPU execution
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 87.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


restart your session

In [2]:
#check GPU config
!nvidia-smi

Sun Aug  4 08:25:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              44W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import userdata
# Defined in the secrets tab in Google Colab (key icon)
hf_token = userdata.get('HF_TOKEN')

### Function to test model

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "jpacifico/Chocolatine-14B-Instruct-4k-DPO",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("jpacifico/Chocolatine-14B-Instruct-4k-DPO")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-medium-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

In [5]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [6]:
def generate_response(prompt, max_new_tokens=500, temperature=0.0):
    messages = [
        {"role": "system", "content": "Tu es un assistant IA baptisé Chocolatine. Ta mission est de fournir des informations sûres, éthiques et précises à l'utilisateur."},
        {"role": "user", "content": prompt},
    ]
    pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    )

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "return_full_text": False,
        "temperature": temperature,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

In [7]:
user_prompt = "Donne moi la recette de la ratatouille."
print(generate_response(user_prompt))

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 Pour préparer une ratatouille traditionnelle française, suivez cette recette simple :

Ingrédients :
- 2 aubergines moyennes
- 3 courgettes moyennes
- 2 poivrons (un rouge, un jaune)
- 4 tomates mûres
- 2 oignons
- 4 gousses d'ail
- Huile d'olive
- Thym
- Laurier
- Sel et poivre

Étapes :

1. Lavez tous les légumes. Coupez les aubergines et les courgettes en dés de taille moyenne. Épépinez et coupez les poivrons en lanières. Hachez les oignons et l'ail.

2. Dans une grande poêle ou une cocotte, faites chauffer de l'huile d'olive à feu moyen. Ajoutez les oignons hachés et faites-les revenir jusqu'à ce qu'ils soient translucides.

3. Ajoutez l'ail écrasé et les poivrons. Faites revenir quelques minutes.

4. Incorporez les aubergines et les courgettes. Laissez cuire pendant environ 10 minutes en remuant de temps en temps, jusqu'à ce que les légumes commencent à ramollir.

5. Ajoutez les tomates hachées, le thym, et un ou deux feuilles de laurier. Salez et poivrez selon votre goût.

6. Co

In [8]:
def stream_response(prompt, max_new_tokens=500, temperature=0.0):
    messages = [
        {"role": "system", "content": "Tu es un assistant IA baptisé Chocolatine. Ta mission est de fournir des informations sûres, éthiques et précises à l'utilisateur."},
        {"role": "user", "content": prompt},
    ]
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "do_sample": False,
        "streamer": streamer,
    }

    pipe(messages, **generation_args)

In [9]:
user_prompt = "Donne moi la recette de la ratatouille."
stream_response(user_prompt)

Pour préparer une ratatouille traditionnelle française, suivez cette recette simple :

Ingrédients :
- 2 aubergines moyennes
- 3 courgettes moyennes
- 2 poivrons (un rouge, un jaune)
- 4 tomates mûres
- 2 oignons
- 4 gousses d'ail
- Huile d'olive
- Thym
- Laurier
- Sel et poivre

Étapes :

1. Lavez tous les légumes. Coupez les aubergines et les courgettes en dés de taille moyenne. Épépinez et coupez les poivrons en lanières. Hachez les oignons et l'ail.

2. Dans une grande poêle ou une cocotte, faites chauffer de l'huile d'olive à feu moyen. Ajoutez les oignons hachés et faites-les revenir jusqu'à ce qu'ils soient translucides.

3. Ajoutez l'ail écrasé et les poivrons. Faites revenir quelques minutes.

4. Incorporez les aubergines et les courgettes. Laissez cuire pendant environ 10 minutes en remuant de temps en temps, jusqu'à ce que les légumes commencent à ramollir.

5. Ajoutez les tomates hachées, le thym, et un ou deux feuilles de laurier. Salez et poivrez selon votre goût.

6. Cou